In [4]:
from scipy import sparse
import time
import nltk
from collections import Counter
import pandas as pd

import csv
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize,blankline_tokenize
from nltk import PorterStemmer, LancasterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import re
import string
from collections import Counter
import json
import re as regex

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV


In [5]:
class TwitterData_Initialize():
    data = []
    processed_data = []
    wordlist = []

    featureList = []
    fea_vect=[]
    
    data_model = None
    data_labels = None
    is_testing = False
    
    def initialize(self, csv_file, is_testing_set=False, from_cached=None):
        if from_cached is not None:
            #self.data_model = pd.read_csv(from_cached)
            self.data_model = pd.read_json(from_cached)

            return

        self.is_testing = is_testing_set

        if not is_testing_set:
            #self.data = pd.read_csv(csv_file, header=0, names=["id", "emotion", "text"])
            self.data = pd.read_json(csv_file)

            #self.data = self.data[self.data["emotion"].isin(["positive", "negative", "neutral"])]
        

        self.processed_data = self.data
        self.wordlist = []
        self.data_model = None
        self.data_labels = None
        
    
    
    
    def do_process(self):
        start_time = time.time()
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"]))
            return row
    
        def tokenize_grams(row):
                #remove numbers
                def remove_numbers(text):
                    return re.sub(r'\d+', '', text)
             
                def to_lower(text):
                    return text.lower()

                #20 Function to remove whitespace
                def remove_whitespace(text):
                    return " ".join(text.split())

                # Function to remove punctuations
                def remove_punctuations(text):
                    text=re.sub(r'[?|*|.|!|+|-]',r'',text)
                    words = nltk.word_tokenize(text)
                    punt_removed = [w for w in words if w.lower() not in string.punctuation]
                    return " ".join(punt_removed)
#30

                # Function to remove stop words
                def remove_stopwords(text, lang='english'):
                        

                    stop_words = set(stopwords.words('english'))
                    word_tokens = word_tokenize(text)
                    #filtered_sentence = [w for w in word_tokens if ((not w in stop_words) or (w in whitelist))]
                    filtered_sentence = []
                    for w in word_tokens:
                        if (w not in stop_words):
                            filtered_sentence.append(w)
                
                    ch=" ".join(filtered_sentence)

                    return ch
#40




                # Function to extract n-grams from text
                def get_ngrams(text, n):

                    n_grams = ngrams(nltk.word_tokenize(text), n)
                    list_grams=[ ' '.join(grams) for grams in n_grams]
                    return list_grams


                # Function to apply lemmatization to a list of words
                def words_lemmatizer(text, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    words = nltk.word_tokenize(text)
                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return " ".join(lemma_words)

                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
    
            
                #convert to string
                idx=row["spans"]
                #ch="".join(x for x in idx if x)
                ch=' '.join(idx)
               
    
                
                
                #words_stemmer(ch, type="PorterStemmer", lang="english", encoding="utf8")
                #Convert to lower case
                txt_num=remove_numbers(ch)

                txt_low=to_lower(txt_num)
                
                txt_ws=remove_whitespace(txt_low)
    

                txt_pun=remove_punctuations(txt_ws)

                #txt_final=remove_stopwords(txt_pun)

                texte=words_lemmatizer(txt_pun)

                n_grams=get_ngrams(texte, 1)

                row["token_spans"] = n_grams
    
                return row
        #self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        print("--- %s seconds ---" % (time.time() - start_time))

    

    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                       ):
        def get_tweets(data):
            start_time = time.time()

            tweets=list(data.processed_data.token_spans)
    
            list_tweet=[]
            for i,chaine in enumerate(tweets):
                ch=" ".join(chaine)
                list_tweet.append(ch)
            print("--- %s seconds ---" % (time.time() - start_time))
    
            return list_tweet
    
        def get_top_n_words(corpus, n=None):
   
            vec = CountVectorizer().fit(corpus)
            bag_of_words = vec.transform(corpus)
            sum_words = bag_of_words.sum(axis=0) 
            words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
            words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
            return words_freq[:n]
        #whitelist=["n't","not","below"]
        whitelist=["to","on","for","up","below","short","long"]
        self.wordlist = []
        """
        corpus=get_tweets(self)
        common_words = get_top_n_words(corpus)
        word_df = pd.DataFrame(data={"word": [k for k, v in common_words],
                                     "occurrences": [v for k, v in common_words]},
                               columns=["word", "occurrences"])
        
        
        word_df.to_csv("wordlist_count.csv", index_label="idx")
        self.wordlist = [k for k, v in common_words]
        
        
        """
        words = Counter()
      
        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]
        

        
                
        print(words.most_common())
        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_copie.csv", index_label="idx")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
        
        
    def build_data_model(self):
        label_column = []
        Id_column=["ID"]
        label_column = ["label"]

        columns = Id_column + label_column + list(
            map(lambda w: w ,self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []

            if True:
                # add label
                current_label = self.processed_data.loc[idx, "sentiment score"]
                current_id = self.processed_data.loc[idx, "id"]
                
                labels.append(current_id)
                labels.append(current_label)
                
                current_row.append(current_id)
                current_row.append(current_label)

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        self.data_labels = pd.Series(labels)
        return self.data_model, self.data_labels


In [6]:
class TwitterData_Initialize_test(TwitterData_Initialize):

    
    
    def do_process(self):
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"].split()))
            return row
    
        def tokenize_grams(row):
                #remove numbers
                def remove_numbers(text):
                    return re.sub(r'\d+', '', text)

                def to_lower(text):
                    return text.lower()
                
                #20 Function to remove whitespace
                def remove_whitespace(text):
                    return " ".join(text.split())

                # Function to remove punctuations
                def remove_punctuations(text):
                    text=re.sub(r'[?|*|.|!|+|-]',r'',text)

                    words = nltk.word_tokenize(text)
                    punt_removed = [w for w in words if w.lower() not in string.punctuation]
                    return " ".join(punt_removed)
#30

                # Function to remove stop words
                def remove_stopwords(text, lang='english'):

                    stop_words = set(stopwords.words('english'))
                    word_tokens = word_tokenize(text)
                    #filtered_sentence = [w for w in word_tokens if ((not w in stop_words) or (w in whitelist))]
                    filtered_sentence = []
                    for w in word_tokens:
                        if (w not in stop_words):
                            filtered_sentence.append(w)
                
                    ch=" ".join(filtered_sentence)

                    return ch
#40




                # Function to extract n-grams from text
                def get_ngrams(text, n):

                    n_grams = ngrams(nltk.word_tokenize(text), n)
                    list_grams=[ ' '.join(grams) for grams in n_grams]
                    return list_grams


                # Function to apply lemmatization to a list of words
                def words_lemmatizer(text, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    words = nltk.word_tokenize(text)
                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return " ".join(lemma_words)

                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
    
            
            
                idx=row["spans"]
                ch=' '.join(idx)


    
                #words_stemmer(ch, type="PorterStemmer", lang="english", encoding="utf8")
                #Convert to lower case
                txt_num=remove_numbers(ch)
                
                
                txt_low=to_lower(txt_num)
                
                txt_ws=remove_whitespace(txt_low)
                    
    
                txt_pun=remove_punctuations(txt_ws)
        
                #txt_final=remove_stopwords(txt_pun)

                texte=words_lemmatizer(txt_pun)

                n_grams=get_ngrams(texte, 1)
                #print("list ngrams--------------\n",n_grams)

                row["token_spans"] = n_grams
    
                return row
        #self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        
    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                      ):
        def get_tweets(data):
            start_time = time.time()

            tweets=list(data.processed_data.token_spans)
    
            list_tweet=[]
            for i,chaine in enumerate(tweets):
                ch=" ".join(chaine)
                list_tweet.append(ch)
            print("--- %s seconds ---" % (time.time() - start_time))
    
            return list_tweet
    
        def get_top_n_words(corpus, n=None):
   
            vec = CountVectorizer().fit(corpus)
            bag_of_words = vec.transform(corpus)
            sum_words = bag_of_words.sum(axis=0) 
            words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
            words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

            return words_freq[:n]

        whitelist=["to","on","for","up","below","short","long"]
        self.wordlist = []
        """
        corpus=get_tweets(self)
        common_words = get_top_n_words(corpus)
        word_df = pd.DataFrame(data={"word": [k for k, v in common_words],
                                     "occurrences": [v for k, v in common_words]},
                               columns=["word", "occurrences"])
        
        
        word_df.to_csv("wordlist_count_test.csv", index_label="idx")
        self.wordlist = [k for k, v in common_words]
        
        
        
        """

        words = Counter()
      
        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]
        

        
                
        print(words.most_common())

        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_test.csv", index_label="idx")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
        
        
    def build_data_model(self):
        
        label_id = ["ID"]

        columns = label_id + list(
            map(lambda w: w ,self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []
            if True:
                # add label
                current_id = self.processed_data.loc[idx, "id"]
                
                labels.append(current_id)
                
                current_row.append(current_id)

            

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        return self.data_model


In [7]:
data = TwitterData_Initialize()
data.initialize("Microblog_Trainingdata.json")

data.processed_data.head(20)

,cashtag,id,sentiment score,source,spans
0,$FB,719659409228451840,0.366,twitter,[watching for bounce tomorrow]
1,$LUV,719904304207962112,0.638,twitter,[record number of passengers served in 2015]
2,$NFLX,5329774,-0.494,stocktwits,[out $NFLX -.35]
3,$DIA,719891468173844480,0.460,twitter,"[Looking for a strong bounce, Lunchtime rally ..."
4,$PLUG,20091246,0.403,stocktwits,[Very intrigued with the technology and growth...
5,$GMCR,5819749,0.000,stocktwits,"[short worked, puts up]"
6,$IBM,709741154393133056,-0.296,twitter,[overbought]
7,$JOSB,17892972,-0.546,stocktwits,"[absolute garbage still up, stores TOTALLY EMP..."
8,$CSTM,709834259687710720,-0.438,twitter,[Biggest Market Losers]
9,$PYPL,708481442079068160,0.408,twitter,[Love this company long time.]


In [8]:
data.do_process()
data.processed_data.head()
data.processed_data

--- 318.39482259750366 seconds ---


,cashtag,id,sentiment score,source,spans,token_spans
0,$FB,719659409228451840,0.366,twitter,[watching for bounce tomorrow],"[watch, for, bounce, tomorrow]"
1,$LUV,719904304207962112,0.638,twitter,[record number of passengers served in 2015],"[record, number, of, passenger, serve, in]"
2,$NFLX,5329774,-0.494,stocktwits,[out $NFLX -.35],"[out, nflx]"
3,$DIA,719891468173844480,0.460,twitter,"[Looking for a strong bounce, Lunchtime rally ...","[look, for, a, strong, bounce, lunchtime, rall..."
4,$PLUG,20091246,0.403,stocktwits,[Very intrigued with the technology and growth...,"[very, intrigue, with, the, technology, and, g..."
5,$GMCR,5819749,0.000,stocktwits,"[short worked, puts up]","[short, work, put, up]"
6,$IBM,709741154393133056,-0.296,twitter,[overbought],[overbought]
7,$JOSB,17892972,-0.546,stocktwits,"[absolute garbage still up, stores TOTALLY EMP...","[absolute, garbage, still, up, store, totally,..."
8,$CSTM,709834259687710720,-0.438,twitter,[Biggest Market Losers],"[big, market, loser]"
9,$PYPL,708481442079068160,0.408,twitter,[Love this company long time.],"[love, this, company, long, time]"


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

def get_tweets(data):
    start_time = time.time()

    tweets=list(data.processed_data.token_spans)
    
    list_tweet=[]
    for i,chaine in enumerate(tweets):
        ch=" ".join(chaine)
        list_tweet.append(ch)
    print(type(list_tweet))
    print("--- %s seconds ---" % (time.time() - start_time))

    return list_tweet

def get_count_vector(corpus):
    vect = CountVectorizer()#appel à l'objet counvectorizer
    vect.fit(corpus)#construction du vecteur de mots
    #print("Vocabulary size: {}".format(len(vect.vocabulary_)))
    #print("Vocabulary content:\n {}".format(vect.vocabulary_))
    vect.get_stop_words()
    X = vect.transform(corpus)
    #print("bag_of_words: {}".format(repr(X)))
    #print("Dense representation of bag_of_words:\n{}".format(
    #X.toarray()))
    return X
   
    
#sparse_matrix = sparse.csr_matrix(bag_of_words)
#print("\nSciPy sparse CSR matrix:\n{}".format(sparse_matrix))

In [10]:
cols = ['sentiment score']  + [col for col in data.processed_data if col != 'sentiment score']
data.processed_data=data.processed_data[cols]
data.processed_data.iloc[:,0:]
data.processed_data.iloc[:,0]

0       0.366
1       0.638
2      -0.494
3       0.460
4       0.403
5       0.000
6      -0.296
7      -0.546
8      -0.438
9       0.408
10     -0.398
11     -0.349
12      0.025
13      0.486
14      0.308
15     -0.372
16      0.461
17      0.408
18     -0.699
19      0.495
20      0.306
21     -0.385
22      0.336
23      0.279
24      0.591
25     -0.351
26     -0.514
27     -0.519
28      0.361
29     -0.042
        ...  
1670   -0.385
1671    0.281
1672   -0.720
1673   -0.351
1674    0.365
1675   -0.382
1676   -0.186
1677   -0.409
1678    0.412
1679    0.465
1680    0.362
1681    0.220
1682    0.365
1683   -0.248
1684   -0.152
1685    0.435
1686    0.414
1687    0.471
1688   -0.589
1689   -0.513
1690   -0.581
1691    0.087
1692    0.230
1693    0.813
1694    0.380
1695   -0.126
1696    0.295
1697    0.405
1698    0.296
1699   -0.296
Name: sentiment score, Length: 1700, dtype: float64

In [11]:
"""train_data, test_data, y_train, y_test = train_test_split(data.processed_data.iloc[:,1:], data.processed_data.iloc[:,0],
                                                   train_size=0.7)
corpus=get_tweets(train_data)
len(corpus)"""
corpus=get_tweets(data)
with open("tweets.txt","w") as fichier:
    fichier.writelines("\n".join(corpus))

<class 'list'>
--- 0.0015037059783935547 seconds ---


In [12]:
def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    
    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    print(vec.stop_words_)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]



common_words = get_top_n_words(corpus, 20)
for word, freq in common_words:
    print(word, freq)

set()
be 292
the 249
to 235
in 151
on 133
for 128
stock 127
of 110
up 109
short 96
look 83
and 83
buy 77
it 76
long 74
this 73
today 68
sell 66
high 66
call 60


In [13]:
corpus=get_tweets(data)
lst=get_top_n_words(corpus)
with open("top_words.csv", "w",newline='') as fichier:
    c = csv.writer(fichier,delimiter=",")

    for l in lst:
        c.writerow(l)
    

<class 'list'>
--- 0.0015292167663574219 seconds ---
set()


In [14]:
data_test = TwitterData_Initialize_test()
data_test.initialize("Microblog_Trialdata.json")

data_test.processed_data.head()
data_test.do_process()

In [17]:


X_train=get_count_vector(corpus)
X_train.toarray()
y_train=data.processed_data["sentiment score"].values

In [18]:
import xgboost as xgb
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
params={'colsample_bytree': 1.0, 'gamma': 0.3, 'max_depth': 4, 'min_child_weight': 4, 'subsample': 0.8}
#Fitting XGB regressor 
model = xgb.XGBRegressor(max_depth=4, min_child_weight= 4,gamma=0.3,subsample=0.8,colsample_bytree=0.7)
model.fit(X_train,y_train)
#bst = xgb.train(param, dtrain,)
print (model)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=4, min_child_weight=4, missing=None,
       n_estimators=100, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.8)


In [20]:
test_corpus=get_tweets(data_test)
test_corpus

<class 'list'>
--- 0.0010027885437011719 seconds ---


['put on a little f short',
 'short some',
 'buying opportunity',
 'scale up on long position',
 'it time to sell bank',
 'enter long',
 'picked some up',
 'time to accumulate for a long position far more upside than downside',
 'look for a strong bounce lunchtime rally come',
 'very intrigue with the technology and growth potential',
 'short work put up',
 'big market loser',
 'goog googl would suck',
 'buying sbux on dip',
 'be a short below and be overbought',
 "do n't put on a down little f short"]

In [21]:
common_words = get_top_n_words(test_corpus, 20)
for word, freq in common_words:
    print(word, freq)

set()
short 5
on 4
put 3
up 3
long 3
little 2
some 2
buying 2
position 2
time 2
to 2
for 2
and 2
be 2
opportunity 1
scale 1
it 1
sell 1
bank 1
enter 1


In [26]:
X_test=get_count_vector(test_corpus)
y_test=data.processed_data["sentiment score"].values
X_test.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 

In [27]:
#Predict 
#output=grid.best_estimator_.predict(X_test)
tweet_row=list(data_test.processed_data["spans"])
#old_pred=list(data_test.processed_data["sentiment score"])
old_pred=list(y_test)
output = model.predict(X_test)
#final_df = pd.DataFrame(tweet_row, columns=labels)
final_df = pd.DataFrame()
final_df["tweet"] = tweet_row

final_df["old_pred"] = old_pred

final_df["Prediction"] = output
final_df.to_csv("Output_1.csv",sep=",")
final_df.head(20)

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', '

In [74]:
#test model accuracy
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


print("mean squared error:" ,mean_squared_error(output, y_test))
print("R2 score:" ,r2_score(output,y_test))

NameError: name 'output' is not defined